In [7]:
google_colab = False

if google_colab:
    !pip install geopandas
    from google.colab import drive
    drive.mount("/content/drive")
    os.chdir('/content/drive/MyDrive/ASU - Zhiang/Projects/instance segmentation/')
    os.listdir()
    
import os

# generate random ellipses

In [2]:
from rsisa.ellipse_instance_generation import spawn_ellipses

In [3]:
help(spawn_ellipses)

Help on function spawn_ellipses in module rsisa.ellipse_instance_generation:

spawn_ellipses(tile_length, ellipse_length_range, density, tile_number, overlap, shp_path, generate_tiff, xres=0.01, yres=0.01)
    spawn a list of random ellipses and save to a shapefile
    
    Args:
        tile_length (float): tile_length/tile_width in meters
        ellipse_length_range (tuple): (ellipse_min, ellipse_max)
        density (int): average number of ellipses per tile
        tile_number (int): number of tiles in horizontal or vertical direction
        overlap (float): overlap between adjacent tiles
        shp_path (string): shape file directory 
        generate_tiff (bool): generate fake tiff file for the boundary of the study area if True. otherwise, not. 
        xres (float): raster resolution in meters. xres and yres are used only when generate_tiff is True
        yres (float): raster resolution in meters



In [5]:
# you need to manually create a folder first: 
#'/root/rsisa/data/random_generation_tutorial'

spawn_ellipses(tile_length=10, 
               ellipse_length_range=(0.3, 1.0), 
               density=100, 
               tile_number=3, 
               overlap=1, 
               shp_path='/root/rsisa/data/random_generation_tutorial', 
               generate_tiff=True,
               xres=0.01,
               yres=0.01)


generate ellipse instances: 


100%|███████████████████████████████████████| 900/900 [00:00<00:00, 5346.79it/s]


In [8]:
# check the generated files
os.listdir('/root/rsisa/data/random_generation_tutorial')

['ellipses_100_10_3_1000_10_300_1000.dbf',
 'ellipses_100_10_3_1000_10_300_1000.cpg',
 'ellipses_100_10_3_1000_10_300_1000.shx',
 'ellipses_100_10_3_1000_10_300_1000.prj',
 'ellipses_100_10_3_1000_10_300_1000.tif',
 'ellipses_100_10_3_1000_10_300_1000.shp']

The names have the following format:  
`ellipses_{density}_{tile_length_in_m}_{tile_number}_{overlap_in_mm}_{pixel_size_in_mm}_{ellipse_max_in_mm}_{ellipse_min_in_mm}`


# Annotation map splitting

In [9]:
from rsisa.annotation_map_split import Tile_Splitter
from rsisa.annotation_map_split import Dataset
from fiona.crs import CRS

In [12]:
help(Tile_Splitter)

Help on class Tile_Splitter in module rsisa.annotation_map_split:

class Tile_Splitter(builtins.object)
 |  Tile_Splitter(shapefile_path, save_dir, crs, area_x1, area_y1, area_x2, area_y2, tile_size, overlap, tif_path, keep_instance_tif=True)
 |  
 |  Methods defined here:
 |  
 |  __init__(self, shapefile_path, save_dir, crs, area_x1, area_y1, area_x2, area_y2, tile_size, overlap, tif_path, keep_instance_tif=True)
 |      Split an annotation map to annotation tiles (shapefiles). It provides an option to split the corresponding tif map. 
 |      
 |      Args:
 |          shapefile_path (string): path of the shapefile to be split
 |          save_dir (string): path of the split shapefiles to be saved
 |          crs (_type_): coordinate reference system (e.g., from fiona.crs import from_epsg)
 |          area_x1 (float): area small coordinate x in meters. Not required if tif_path is not None.
 |          area_y1 (float): area small coordinate y in meters.
 |          area_x2 (float): a

In [40]:
shp_path = '/root/rsisa/data/random_generation_tutorial/ellipses_100_10_3_1000_10_300_1000.shp'
split_shp_tif_dir = '/root/rsisa/data/random_generation_tutorial/split_shp_tif'
tif_path = '/root/rsisa/data/random_generation_tutorial/ellipses_100_10_3_1000_10_300_1000.tif'
save_sub_dir = '/root/rsisa/data/random_generation_tutorial'
tile_length = 10
overlap = 1
pixel_size = 0.01
tile_number = 3
area_x2 = (tile_length - overlap) * tile_number + overlap
area_y2 = (tile_length - overlap) * tile_number + overlap 
ellipse_max = 1.0


epsg = 32611  # WGS 84 / UTM zone 11N
crs = CRS.from_epsg(epsg)

print(shp_path)
print(split_shp_tif_dir)
print(tif_path)
print(save_sub_dir)
print(area_x2)
print(area_y2)

/root/rsisa/data/random_generation_tutorial/ellipses_100_10_3_1000_10_300_1000.shp
/root/rsisa/data/random_generation_tutorial/split_shp_tif
/root/rsisa/data/random_generation_tutorial/ellipses_100_10_3_1000_10_300_1000.tif
/root/rsisa/data/random_generation_tutorial
28
28


In [27]:
# split the study area and create annotation tiles
split_config = {'shapefile_path': shp_path, 
                'save_dir': split_shp_tif_dir, 
                'crs':crs, 
                'area_x1':0, 
                'area_y1':0, 
                'area_x2': area_x2,
                'area_y2': area_y2,
                'tile_size': tile_length, 
                'overlap': overlap,
                'tif_path': tif_path, 
                'keep_instance_tif': True}

ts = Tile_Splitter(**split_config)
ts.split()


# crop the study area (remove edges)
split_config = {'shapefile_path': shp_path, 
                'save_dir': save_sub_dir, # this should be sub dir
                'crs':crs, 
                'area_x1':0, 
                'area_y1':0, 
                'area_x2':area_x2,
                'area_y2':area_y2, 
                'tile_size':(tile_length - overlap) * tile_number + overlap + ellipse_max/2., # just change this to the size of study area. here because the ellipses go slightly than the study area, if we set tile_size of 30, the exceeding part will be split into other tiles. thus we have a large tile_size here.
                'overlap':0,
                'tif_path': None, 
                'keep_instance_tif': True}

ts = Tile_Splitter(**split_config)
ts.split()

annotation splitting ... 
annotation splitting done 
save tile shapefiles: 


100%|███████████████████████████████████████████| 16/16 [00:00<00:00, 59.33it/s]


save tiff tiles: 


100%|███████████████████████████████████████████| 16/16 [00:00<00:00, 65.18it/s]


annotation splitting ... 
annotation splitting done 
save tile shapefiles: 


100%|█████████████████████████████████████████████| 1/1 [00:00<00:00,  7.73it/s]


In [28]:
# check the generated files
os.listdir('/root/rsisa/data/random_generation_tutorial')

['ellipses_100_10_3_1000_10_300_1000_time_space.npy',
 'ellipses_100_10_3_1000_10_300_1000.dbf',
 'ellipses_100_10_3_1000_10_300_1000.cpg',
 'split_shp_tif',
 '0_0.cpg',
 'ellipses_100_10_3_1000_10_300_1000.shx',
 'ellipses_100_10_3_1000_10_300_1000.prj',
 '0_0.prj',
 '0_0.shx',
 '0_0.shp',
 'ellipses_100_10_3_1000_10_300_1000.tif',
 '0_0.dbf',
 'ellipses_100_10_3_1000_10_300_1000.shp']

In [42]:
# create prediction tiles using the annotation tiles
# help(Dataset)
ellipse_data = Dataset(pixel_size=int(tile_length/pixel_size), split_path=split_shp_tif_dir, input_channel=(0,)) 
ellipse_data.save_pickles(zip=zip)

save pickles: 


100%|███████████████████████████████████████████| 16/16 [00:17<00:00,  1.07s/it]


In [47]:
# check generated prediction tiles (.zip files)
os.listdir('/root/rsisa/data/random_generation_tutorial/split_shp_tif')[:20]

['2_3.shp',
 '0_3.dbf',
 '3_1.shp',
 '3_2.cpg',
 '1_0.cpg',
 '1_1.shp',
 '1_3.dbf',
 '2_2.cpg',
 '1_1.tif',
 '3_3.prj',
 '1_2.tif',
 '0_1.dbf',
 'ellipses_100_10_3_1000_10_300_1000_time_space.npy',
 '1_2.dbf',
 '3_3.shp',
 '3_0.shx',
 '3_1.tif',
 '2_1.prj',
 '2_1.tif',
 '3_0.dbf']

`ellipses_100_10_3_1000_10_300_1000_time_space.npy` logs the time and space usage of the annotation splitting process

# Instance registration

In [48]:
from rsisa.instance_registration import Instance_Registration

In [49]:
help(Instance_Registration)

Help on class Instance_Registration in module rsisa.instance_registration:

class Instance_Registration(builtins.object)
 |  Instance_Registration(instance_dir, save_shapefile, tif_height_pixel=1000, tif_width_pixel=1000, tif_height_res=-0.01, tif_width_res=0.01, tile_overlap_ratio=0.1, detection_threshold=0.75, segmentation_threshold=0.5, iou_threshold=0.5, disable_merge=False, test=True, unzip=False)
 |  
 |  Methods defined here:
 |  
 |  __init__(self, instance_dir, save_shapefile, tif_height_pixel=1000, tif_width_pixel=1000, tif_height_res=-0.01, tif_width_res=0.01, tile_overlap_ratio=0.1, detection_threshold=0.75, segmentation_threshold=0.5, iou_threshold=0.5, disable_merge=False, test=True, unzip=False)
 |      merge and register instances using split pickle files and tiff tiles. The pickle file name is consistant with the names of its corresponding shapefile and tif file. Each pickle file includes a dictionary of instance predictions, e.g., {'image'=, 'bb'=, 'labels'=, 'scores'

In [53]:
instance_dir = "/root/rsisa/data/random_generation_tutorial/split_shp_tif"
save_shp_file = "/root/rsisa/data/random_generation_tutorial/ellipses_100_10_3_1000_10_300_1000_merge.shp"
iou_threshold = 0.75

print(instance_dir)
print(save_shp_file)

/root/rsisa/data/random_generation_tutorial/split_shp_tif
/root/rsisa/data/random_generation_tutorial/ellipses_100_10_3_1000_10_300_1000_merge.shp


In [54]:
ir = Instance_Registration(instance_dir, 
                           save_shp_file,
                           tif_width_pixel=int(tile_length/pixel_size),
                           tif_height_pixel=int(tile_length/pixel_size),
                           tif_width_res=pixel_size,
                           tif_height_res=-pixel_size,
                           tile_overlap_ratio=overlap/tile_length,
                           iou_threshold=iou_threshold, 
                           disable_merge=False, 
                           unzip=zip)


updated_tile_files, timestamps = ir.start_registration()
ir.combine_shapefiles()

Instance registration: 


100%|███████████████████████████████████████████| 16/16 [01:15<00:00,  4.73s/it]


In [55]:
# check the generated files
os.listdir('/root/rsisa/data/random_generation_tutorial')

['ellipses_100_10_3_1000_10_300_1000_time_space.npy',
 'ellipses_100_10_3_1000_10_300_1000_merge.dbf',
 'ellipses_100_10_3_1000_10_300_1000.dbf',
 'ellipses_100_10_3_1000_10_300_1000.cpg',
 'ellipses_100_10_3_1000_10_300_1000_merge.shp',
 'split_shp_tif',
 '0_0.cpg',
 'ellipses_100_10_3_1000_10_300_1000.shx',
 'ellipses_100_10_3_1000_10_300_1000_merge.shx',
 'ellipses_100_10_3_1000_10_300_1000_merge.cpg',
 'ellipses_100_10_3_1000_10_300_1000.prj',
 '0_0.prj',
 '0_0.shx',
 '0_0.shp',
 'ellipses_100_10_3_1000_10_300_1000.tif',
 'ellipses_100_10_3_1000_10_300_1000_merge_time_space.npy',
 'ellipses_100_10_3_1000_10_300_1000_merge.prj',
 '0_0.dbf',
 'ellipses_100_10_3_1000_10_300_1000.shp']

`ellipses_100_10_3_1000_10_300_1000_merge_time_space.npy` logs the time and space usage of the instance registration process

# Evaluation

In [56]:
from rsisa.evaluation import evaluate

# note that the ground truth should be cropped shapefile instead of the original generated shapefile
evaluate('/root/rsisa/data/random_generation_tutorial/ellipses_100_10_3_1000_10_300_1000.shp', 
         '/root/rsisa/data/random_generation_tutorial/ellipses_100_10_3_1000_10_300_1000_merge.shp')

total:  906
TP, TN, FP, FN:  255 629 7 15
FP:  [223, [79, 79, 805], 201, 897, 460, 255, 249]
FN:  [540, 706, 24, 430, 777, 155, 292, 558, 767, 67, 188, 291, 643, 145, 729]
Duplicated ids:  []
accuracy: 97.57%
precision: 97.33%
recall: 94.44%
